# Credit Risk - Probability of Default

## Importing the libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk

## Importing data from a csv

In [3]:
pd.options.display.max_columns = None

In [6]:
df_loan = pd.read_csv('/data/loan_data_2007_2014.csv')
df_loan.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13371 entries, 0 to 13370
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   13371 non-null  int64  
 1   id                           13371 non-null  int64  
 2   member_id                    13371 non-null  int64  
 3   loan_amnt                    13371 non-null  int64  
 4   funded_amnt                  13371 non-null  int64  
 5   funded_amnt_inv              13371 non-null  float64
 6   term                         13371 non-null  object 
 7   int_rate                     13371 non-null  float64
 8   installment                  13371 non-null  float64
 9   grade                        13371 non-null  object 
 10  sub_grade                    13371 non-null  object 
 11  emp_title                    12498 non-null  object 
 12  emp_length                   12900 non-null  object 
 13  home_ownership  

Changing variables types

In [8]:
df_loan['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [24]:
df_loan['emp_length_int'] = df_loan['emp_length'].str.replace("\+ years", "")
df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\< 1 year", str(0))
df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace('n/a', str(0))
df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\ years", "")
df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\ year", "")

<ipython-input-24-4110cd9adc17>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_loan['emp_length_int'] = df_loan['emp_length'].str.replace("\+ years", "")
<ipython-input-24-4110cd9adc17>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\< 1 year", str(0))
<ipython-input-24-4110cd9adc17>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\ years", "")
<ipython-input-24-4110cd9adc17>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_loan['emp_length_int'] = df_loan['emp_length_int'].str.replace("\ year", "")


In [28]:
df_loan['emp_length_int'].fillna(0, inplace=True)
df_loan['emp_length_int'].unique()

array(['10', '0', '1', '3', '8', '9', '4', '5', '6', '2', '7', 0],
      dtype=object)

In [29]:
# Numbers still strings
df_loan['emp_length_int'] = pd.to_numeric(df_loan['emp_length_int'])
df_loan['emp_length_int'].unique()

array([10,  0,  1,  3,  8,  9,  4,  5,  6,  2,  7])

In [30]:
df_loan['emp_length_int'].dtypes 

dtype('int64')

Variable 'Employment length in years' now is a integer number, going to another variable.

In [ ]:
df_loan['earliest_cr_line'].unique() #should be a date variable

In [39]:
df_loan['earliest_cr_line_date'] = pd.to_datetime(df_loan['earliest_cr_line'], format='%b-%y')
df_loan['mths_since_earliest_cr_line'] = round( pd.to_numeric( (pd.to_datetime('2017-12-01') - df_loan['earliest_cr_line_date'] ) / np.timedelta64(1, 'M') ))
df_loan['mths_since_earliest_cr_line'].describe()

count    13370.000000
mean       243.600898
std         91.851954
min       -609.000000
25%        191.000000
50%        229.000000
75%        286.000000
max        586.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [46]:
df_loan.loc[ : , ['earliest_cr_line','earliest_cr_line_date','mths_since_earliest_cr_line']][df_loan['mths_since_earliest_cr_line']<0]

,earliest_cr_line,earliest_cr_line_date,mths_since_earliest_cr_line
1580,Sep-62,2062-09-01,-537.0
1770,Sep-68,2068-09-01,-609.0
2799,Sep-64,2064-09-01,-561.0
3282,Sep-67,2067-09-01,-597.0
3359,Feb-65,2065-02-01,-566.0
3413,Jun-67,2067-06-01,-594.0
3607,Aug-67,2067-08-01,-596.0
3989,Dec-63,2063-12-01,-552.0
4440,Sep-68,2068-09-01,-609.0
4449,Sep-68,2068-09-01,-609.0
